<a href="https://colab.research.google.com/github/ZaneZaiontz/YosAI/blob/main/YosAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

Opening and reading in data

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
from keras.preprocessing import sequence
import keras
import numpy as np
import os
import random as rand

def readFiles():
    file = open('./zhaoClean.txt', 'r')
    lines = file.readlines()
    file.close()
    file = open('./zhaoClean.txt', 'r')
    return file.read().lower(), lines

text, lines = readFiles()

Encoding


In [ ]:
# training length
sequenceLen = 64
vocab = sorted(set(text))
# Unique ID 
charToInt = {j:i for i, j in enumerate(vocab)}
intToChar = np.array(vocab)

examples_per_epoch = len(text)//(sequenceLen+1)
tmpData = np.array([charToInt[i] for i in text])
charData = tf.data.Dataset.from_tensor_slices(tmpData)
sequences = charData.batch(sequenceLen+1, drop_remainder=True)

def splitInput(chunk):  
    inText = chunk[:-1]  
    toText = chunk[1:] 
    return inText, toText 

dataset = sequences.map(splitInput)  

Build Model/Shuffle Data

In [ ]:
sizeOfEmbed = 256
sizeOfRNN = 1024
sizeOfBuff = 10000
sizeofBatch = 64
sizeOfVocab = len(vocab)
data = dataset.shuffle(sizeOfBuff).batch(sizeofBatch, drop_remainder=True)

def buildTheModel(vocabSize, embedSize, rnnSize, batchSize):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocabSize, embedSize, batch_input_shape=[batchSize, None]),
    tf.keras.layers.LSTM(rnnSize, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocabSize)])
  return model
model = buildTheModel(sizeOfVocab, sizeOfEmbed, sizeOfRNN, sizeofBatch)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           15360     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 60)            61500     
Total params: 5,323,836
Trainable params: 5,323,836
Non-trainable params: 0
_________________________________________________________________


Compile/Loss Function


Checkpoints/Fitting

In [ ]:
numOfEpochs = 32

def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
model.compile(optimizer='adam', loss=loss)

checkLocation = './trainingData'
checkPre = os.path.join(checkLocation, "CheckpointData_{epoch}")

checkBack=tf.keras.callbacks.ModelCheckpoint(filepath=checkPre, save_weights_only=True)
history = model.fit(data, epochs=numOfEpochs, callbacks=[checkBack])

model = buildTheModel(sizeOfVocab, sizeOfEmbed, sizeOfRNN, batchSize=1)
model.load_weights(tf.train.latest_checkpoint(checkLocation))
model.build(tf.TensorShape([1, None]))

Epoch 1/32
61/61 [==============================] - 6s 45ms/step - loss: 3.2218
Epoch 2/32
61/61 [==============================] - 3s 45ms/step - loss: 2.5569
Epoch 3/32
61/61 [==============================] - 3s 46ms/step - loss: 2.3243
Epoch 4/32
61/61 [==============================] - 3s 46ms/step - loss: 2.1728
Epoch 5/32
61/61 [==============================] - 3s 47ms/step - loss: 2.0658
Epoch 6/32
61/61 [==============================] - 3s 47ms/step - loss: 1.9725
Epoch 7/32
61/61 [==============================] - 3s 48ms/step - loss: 1.8792
Epoch 8/32
61/61 [==============================] - 3s 48ms/step - loss: 1.8026
Epoch 9/32
61/61 [==============================] - 3s 48ms/step - loss: 1.7392
Epoch 10/32
61/61 [==============================] - 3s 48ms/step - loss: 1.6875
Epoch 11/32
61/61 [==============================] - 3s 47ms/step - loss: 1.6367
Epoch 12/32
61/61 [==============================] - 3s 47ms/step - loss: 1.5927
Epoch 13/32
61/61 [==================

Output

In [ ]:
def generate_text(model):
  newL = 0

  startInt = (rand.randint(0, len(lines)))/3
  startLine = lines[int(startInt)]
  startLine = startLine.lower()

  input_eval = [charToInt[s] for s in startLine]
  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []

  model.reset_states()
  while newL < 3:
      predictions = model(input_eval)

      predictions = tf.squeeze(predictions, 0)
      predictions = predictions
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
      
      input_eval = tf.expand_dims([predicted_id], 0)

      if (intToChar[predicted_id] == '\n'):
        newL += 1
      if (newL >= 3):
        text_generated.append(intToChar[predicted_id])
        break

      text_generated.append(intToChar[predicted_id])

  return (''.join(text_generated))

print(generate_text(model))



between snows fade
intifaves
at a cigame lanour him

